In [ ]:
#k center problem with outliers
import numpy as np
import matplotlib.pyplot as plt
import math 

class KCenterProblemWithOutliers():
  def __init__(self, x_input, y_input) -> None:
    self.x_input, self.y_input = x_input, y_input
    self.NUM_INPUTS = len(x_input)
    self.dists = [[0 for _ in range(self.NUM_INPUTS)] for _ in range(self.NUM_INPUTS)]
    self.selected_samples = []

  def compute_dists(self):
    for r in range(self.NUM_INPUTS):
      for c in range(self.NUM_INPUTS):
        if not r == c:
          self.dists[r][c] = math.sqrt((self.x_input[r] - self.x_input[c])**2 + (self.y_input[r] - self.y_input[c])**2)

  def select_samples(self, query_size):
    def maxindex(dist):
      max_ind = 0
      for i in range(self.NUM_INPUTS):
          if (dist[i] > dist[max_ind]):
              max_ind = i
      return max_ind

    self.compute_dists()

    self.center_dists = [float('inf') for _ in range(self.NUM_INPUTS)]

    max_center = 0

    for _ in range(query_size):
      self.selected_samples.append(max_center)
      
      for j in range(self.NUM_INPUTS):
        self.center_dists[j] = min(self.center_dists[j], self.dists[max_center][j])
      
      max_center = maxindex(self.center_dists)

    x_query, y_query = [], []
    for sample_ind in self.selected_samples:
      x_query.append(self.x_input[sample_ind])
      y_query.append(self.y_input[sample_ind])

    self.plot_points(x_input, y_input, x_query, y_query)

    return self.selected_samples

  def plot_points(self, x_input, y_input, x_query, y_query):
    plt.scatter(x=x_input, y=y_input, color='blue')
    plt.scatter(x=x_query, y=y_query, color='red')
    plt.show()

if __name__ == '__main__' :
  min_val, max_val, num_vals = 0, 100, 100
  query_size = 10

  x_input = np.random.randint(min_val, max_val, num_vals).tolist()
  y_input = np.random.randint(min_val, max_val, num_vals).tolist()

  for _ in range(10):
    if num_vals // query_size > 0:
      query = KCenterProblemWithOutliers(x_input, y_input)

      prev_query_inds = query.select_samples(query_size)
      prev_query_inds.sort(reverse=True)

      for ind in prev_query_inds:
        x_input.pop(ind)
        y_input.pop(ind)

In [ ]:
import torch
import torchplot as plt

num_vals = 10
dim = 5
query_size = 2

inputs = []

for _ in range(num_vals):
  inputs.append(torch.rand(1,dim))

dists_between_inputs = torch.zeros(num_vals, num_vals)

for r in range(num_vals):
  for c in range(num_vals):
    dists_between_inputs[r][c] = torch.cdist(inputs[r], inputs[c])

print(dists_between_inputs)

# x_input = torch.rand(1,dim)
# y_input = torch.rand(1,dim)
# dist = torch.cdist(x_input, y_input)

# print(f'x_input = {x_input}')
# print(f'y_input = {y_input}')
# print(f'dist = {dist}')

# center_dists = torch.Tensor([float('inf') for _ in range(dim)])

# max_center = 0
# selected_sample_inds = []

# def max_ind(dist):
#       max_ind = 0
#       for i in range(num_vals):
#           if (dist[i] > dist[max_ind]):
#               max_ind = i
#       return max_ind

# for _ in range(query_size):
#   selected_sample_inds.append(max_center)

#   for i in range(num_vals):
#     center_dists[i] = min(center_dists[i], dist[max_center][i])

#   max_center = max_ind(center_dists)

# x_query, y_query = [], []
# for sample_ind in selected_sample_inds:
#   x_query.append(x_input[sample_ind])
#   y_query.append(y_input[sample_ind])

# plt.plot(x_input, y_input, '.')
# plt.plot(x_query, y_query, 'x')

In [ ]:
import torch
import torchplot as plt

x1 = torch.Tensor([1,2,3,4,5,6,7,8])
x2 = torch.Tensor([2,5,6,7,9,8,9,9])

dist  = torch.cdist(x1,x2)

plt.plot(x1, x2, '.')